# PingPong Library Tutorial

## Introduction

The PingPong library demonstrates how to build simple communication between Syft clients. It allows users to:

1. **Send pings** to other Syft clients
2. **Receive pings** and automatically respond with pongs
3. **List available datasites** to communicate with

This tutorial will walk you through setting up clients and exchanging pings between them.

### How It Works

PingPong uses Syft's remote procedure call (RPC) functionality to send messages between clients. Each client runs a server component in the background that listens for ping requests and responds with pongs.

## 1. Setup

First, let's import the PingPong library.

In [1]:
import pingpong as pp

## 2. Creating Clients

Let's create two PingPong clients. Each client connects to a Syft instance and runs a server in the background to respond to pings.

### 2.1 Bob's Client

We'll create a client for Bob using his config file. The logs will show the client connecting and starting the background server.

In [2]:
# Create a client for Bob using his config file
bob_client = pp.client("~/.syft_bob_config.json")

2025-03-08 18:52:21.348 | INFO     | pingpong:__init__:46 - 🔑 Connected as: bob@openmined.org
2025-03-08 18:52:21.349 | INFO     | pingpong:_run_server:64 - 🚀 SERVER: Running pong server as bob@openmined.org
2025-03-08 18:52:21.349 | INFO     | pingpong:_start_server:59 - 🔔 PingPong server started for bob@openmined.org
2025-03-08 18:52:21.350 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /ping
2025-03-08 18:52:21.350 | INFO     | pingpong:_run_server:71 - 📡 SERVER: Listening for pings at /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/pingpong/rpc
2025-03-08 18:52:21.353 | INFO     | syft_event.server2:publish_schema:96 - Published schema to /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/pingpong/rpc/rpc.schema.json


### 2.2 Alice's Client

Now let's create a second client for Alice. This will allow us to demonstrate communication between two different Syft instances.

In [3]:
# Create a client for Alice using her config file
alice_client = pp.client("~/.syft_alice_config.json")

2025-03-08 18:53:38.718 | INFO     | pingpong:__init__:46 - 🔑 Connected as: alice@openmined.org
2025-03-08 18:53:38.719 | INFO     | pingpong:_run_server:64 - 🚀 SERVER: Running pong server as alice@openmined.org
2025-03-08 18:53:38.719 | INFO     | pingpong:_start_server:59 - 🔔 PingPong server started for alice@openmined.org
2025-03-08 18:53:38.720 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /ping
2025-03-08 18:53:38.720 | INFO     | pingpong:_run_server:71 - 📡 SERVER: Listening for pings at /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/pingpong/rpc
2025-03-08 18:53:38.722 | INFO     | syft_event.server2:publish_schema:96 - Published schema to /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/pingpong/rpc/rpc.schema.json


## 3. Listing Available Datasites

Before sending a ping, we need to know which datasites (other users) are available to communicate with. The `list_datasites()` method returns all datasites that our client can see.

In [4]:
# List available datasites for Alice
datasites = alice_client.list_datasites()

# Show a summary instead of the full list
print(f"Available datasites for Alice: {len(datasites)} datasites found")
print()
print("Some notable datasites:")
for ds in ['alice@openmined.org', 'bob@openmined.org', 'test@openmined.org', 
           'dave@openmined.org', 'andrew@openmined.org']:
    if ds in datasites:
        print(f"  - {ds}")

Available datasites for Alice: 143 datasites found

Some notable datasites:
  - alice@openmined.org
  - bob@openmined.org
  - test@openmined.org
  - dave@openmined.org
  - andrew@openmined.org


## 4. Sending a Ping

Now let's have Bob send a ping to Alice. The client's `ping()` method takes the target datasite as an argument.

When Bob pings Alice:
1. Bob's client sends a ping request to Alice's server
2. Alice's server receives the ping and automatically responds with a pong
3. Bob's client receives the pong response

Watch the logs to see the message flow:

In [5]:
# Bob pings Alice
response = bob_client.ping("alice@openmined.org")

2025-03-08 18:53:41.934 | INFO     | pingpong:ping:124 - 📤 SENDING: Ping to alice@openmined.org
2025-03-08 18:53:46.863 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/pingpong/rpc/ping/0313874f-f261-425c-b753-5961d6bdddfb.request
2025-03-08 18:53:46.864 | INFO     | pingpong:_pong:102 - 🔔 RECEIVED: Ping request - msg='Hello from bob@openmined.org!' ts=datetime.datetime(2025, 3, 8, 23, 53, 41, 934953, tzinfo=TzInfo(UTC))
2025-03-08 18:53:50.130 | INFO     | pingpong:ping:142 - 📥 RECEIVED: Pong from alice@openmined.org. Time: 8.20s


## 5. Examining the Response

Let's look at the response Bob received from Alice. This is a `PongResponse` object that contains the message and timestamp.

In [6]:
# Examine the response
response

PongResponse(msg='Pong from alice@openmined.org', ts=datetime.datetime(2025, 3, 8, 23, 53, 46, 865060, tzinfo=TzInfo(UTC)))

We can access individual fields of the response:

In [7]:
print(f"Message received: {response.msg}")
print(f"Timestamp: {response.ts}")

Message received: Pong from alice@openmined.org
Timestamp: 2025-03-08 23:53:46.865060+00:00


## 6. Reverse Direction - Alice Pings Bob

Let's try the reverse direction to show bidirectional communication. Now Alice will ping Bob:

In [8]:
# Alice pings Bob
reverse_response = alice_client.ping("bob@openmined.org")

2025-03-08 18:54:10.712 | INFO     | pingpong:ping:124 - 📤 SENDING: Ping to bob@openmined.org
2025-03-08 18:54:11.763 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/pingpong/rpc/ping/f38d9c71-3a1c-4f6a-baa5-9e6a93e0a7b2.request
2025-03-08 18:54:11.765 | INFO     | pingpong:_pong:102 - 🔔 RECEIVED: Ping request - msg='Hello from alice@openmined.org!' ts=datetime.datetime(2025, 3, 8, 23, 54, 10, 712541, tzinfo=TzInfo(UTC))
2025-03-08 18:54:13.942 | INFO     | pingpong:ping:142 - 📥 RECEIVED: Pong from bob@openmined.org. Time: 3.23s


In [9]:
print(f"Message received: {reverse_response.msg}")
print(f"Timestamp: {reverse_response.ts}")

Message received: Pong from bob@openmined.org
Timestamp: 2025-03-08 23:54:11.765352+00:00


## 7. Error Handling

Let's see what happens when we try to ping a non-existent datasite:

In [10]:
# Try to ping a non-existent datasite
error_response = bob_client.ping("nonexistent@openmined.org")

2025-03-08 18:54:25.315 | ERROR    | pingpong:ping:117 - Invalid datasite: nonexistent@openmined.org
2025-03-08 18:54:25.315 | INFO     | pingpong:ping:119 - Available datasites:
2025-03-08 18:54:25.316 | INFO     | pingpong:ping:121 -   - Morganabuell98@gmail.com
2025-03-08 18:54:25.316 | INFO     | pingpong:ping:121 -   - a@gmail.com
2025-03-08 18:54:25.316 | INFO     | pingpong:ping:121 -   - a@openmined.org
2025-03-08 18:54:25.316 | INFO     | pingpong:ping:121 -   - abinvarghese90@gmail.com
2025-03-08 18:54:25.316 | INFO     | pingpong:ping:121 -   - alice@openmined.org


In [11]:
print(f"Response when pinging invalid datasite: {error_response}")

Response when pinging invalid datasite: None


## 8. Cleanup

When we're done, we should properly close the clients to shut down the background servers.

In [12]:
# Close Bob's client
bob_client.close()

2025-03-08 18:53:56.979 | INFO     | pingpong:close:162 - 👋 Shutting down PingPong client...


In [13]:
# Close Alice's client
alice_client.close()

2025-03-08 18:53:59.531 | INFO     | pingpong:close:162 - 👋 Shutting down PingPong client...


## 9. Summary

In this tutorial, we've demonstrated:

1. Creating PingPong clients for different Syft instances
2. Listing available datasites
3. Sending pings between clients
4. Receiving pong responses
5. Handling errors for invalid datasites
6. Properly cleaning up resources

The PingPong library provides a simple example of how to build communication between Syft clients. This pattern can be extended to create more complex distributed applications.

## 10. Additional Information

### Common Issues

- **Connection timeouts**: If pings take too long to respond, the client might time out. The default timeout is 30 seconds.
- **Missing datasites**: Make sure the target datasite exists and your client has permission to access it.
- **Server already running**: If you see watchdog errors about directories already being watched, it means another server is already monitoring that directory.

### Next Steps

- Extend the PingPong library to send custom messages
- Implement more complex RPC calls between clients
- Use this pattern to build distributed applications on Syft